In [149]:
import pandas as pd
import neattext.functions as nfx
import tensorflow as tf
import tensorflow_text as tf_text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
#a=tf.config.list_physical_devices()[1]
#tf.config.experimental.set_memory_growth(a,True)
#tf.config.experimental.set_synchronous_execution(False)

In [41]:
'''df = pd.read_csv("data.txt",header=None)
df_new = df[0].str.split(r'(__label__enhancement)|(__label__bug)|(__label__question)',expand=True)
enh_df = df_new[df_new[1] == '__label__enhancement'][[1,4]]
bug_df = df_new[df_new[2] == '__label__bug'][[2,4]]
question_df = df_new[df_new[3] == '__label__question'][[3,4]]
enh_df.columns = ['label','description']
bug_df.columns = ['label','description']
question_df.columns = ['label','description']
df = pd.concat([enh_df, bug_df, question_df])
df['label'] = df['label'].str.replace('__label__', '')'''

'df = pd.read_csv("data.txt",header=None)\ndf_new = df[0].str.split(r\'(__label__enhancement)|(__label__bug)|(__label__question)\',expand=True)\nenh_df = df_new[df_new[1] == \'__label__enhancement\'][[1,4]]\nbug_df = df_new[df_new[2] == \'__label__bug\'][[2,4]]\nquestion_df = df_new[df_new[3] == \'__label__question\'][[3,4]]\nenh_df.columns = [\'label\',\'description\']\nbug_df.columns = [\'label\',\'description\']\nquestion_df.columns = [\'label\',\'description\']\ndf = pd.concat([enh_df, bug_df, question_df])\ndf[\'label\'] = df[\'label\'].str.replace(\'__label__\', \'\')'

In [42]:
df=pd.read_csv("final_data.csv")
df['description_clean'] = df['description'].apply(lambda x: nfx.remove_stopwords(str(x).lower()))

In [43]:
df

,Unnamed: 0,label,description,description_clean
0,0,enhancement,Exception handling # Description The methods...,exception handling # description methods use t...
1,1,enhancement,Subpasses in MSL and HLSL Sorry for all the i...,subpasses msl hlsl sorry issues 😅 given follow...
2,2,enhancement,Collect new discovered nodes Collect new adde...,collect new discovered nodes collect new added...
3,3,enhancement,Lock property : several properties at once,lock property : properties
4,5,enhancement,Randomize: remove sliders,randomize: remove sliders
...,...,...,...,...
29995,29940,question,小白问题：监控页面配置了密码访问，如何通过http获取View JSON。 如题,小白问题：监控页面配置了密码访问，如何通过http获取view json。 如题
29996,29942,question,Possible to support 1.0.0-* and 2.0.0-rc2-079...,possible support 1.0.0-* 2.0.0-rc2-0792 sql db...
29997,29950,question,Input/output error when getattr() called when...,input/output error getattr() called entered mo...
29998,29983,question,No example ? I don't see any application of ...,example ? application prooph repo committed ?


In [44]:
Xfeatures = df['description_clean']
ylabels = df['label']

In [45]:
Xfeatures

0        exception handling # description methods use t...
1        subpasses msl hlsl sorry issues 😅 given follow...
2        collect new discovered nodes collect new added...
3                               lock property : properties
4                                randomize: remove sliders
                               ...                        
29995             小白问题：监控页面配置了密码访问，如何通过http获取view json。 如题
29996    possible support 1.0.0-* 2.0.0-rc2-0792 sql db...
29997    input/output error getattr() called entered mo...
29998        example ? application prooph repo committed ?
29999    crash resizing grid item hi tiberiu, thank tak...
Name: description_clean, Length: 30000, dtype: object

In [46]:
tokenizer=CountVectorizer()
tokenizer.fit(Xfeatures)
x=tokenizer.transform(Xfeatures)

In [47]:
lable_encoder=LabelEncoder
y=lable_encoder.fit_transform(lable_encoder,ylabels)

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
X_train, X_val,Y_train, Y_val = train_test_split(x, y,test_size=0.2,random_state=10)

In [68]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Input([None,166835]))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(32,activation = 'relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics="accuracy",optimizer=tf.keras.optimizers.SGD(learning_rate=0.1))

In [72]:
model.fit(X_train, Y_train, epochs=13, batch_size=3000, validation_data=(X_val,Y_val))

Epoch 1/13
8/8 [==============================] - 1s 83ms/step - loss: 0.4161 - accuracy: 0.8358 - val_loss: 0.7287 - val_accuracy: 0.7302
Epoch 2/13
8/8 [==============================] - 1s 80ms/step - loss: 0.4095 - accuracy: 0.8388 - val_loss: 0.7729 - val_accuracy: 0.7187
Epoch 3/13
8/8 [==============================] - 1s 81ms/step - loss: 0.4563 - accuracy: 0.8100 - val_loss: 0.7920 - val_accuracy: 0.7088
Epoch 4/13
8/8 [==============================] - 1s 82ms/step - loss: 0.3975 - accuracy: 0.8424 - val_loss: 0.7372 - val_accuracy: 0.7367
Epoch 5/13
8/8 [==============================] - 1s 82ms/step - loss: 0.4107 - accuracy: 0.8347 - val_loss: 0.8793 - val_accuracy: 0.6857
Epoch 6/13
8/8 [==============================] - 1s 84ms/step - loss: 0.3860 - accuracy: 0.8501 - val_loss: 0.7330 - val_accuracy: 0.7385
Epoch 7/13
8/8 [==============================] - 1s 74ms/step - loss: 0.3853 - accuracy: 0.8492 - val_loss: 0.8655 - val_accuracy: 0.6920
Epoch 8/13
8/8 [===========

In [61]:
vocab=tokenizer.vocabulary_

In [14]:
len(vocab)

166835

In [15]:
vocab

{'exception': 80305,
 'handling': 91827,
 'description': 72121,
 'methods': 109876,
 'use': 152548,
 'try': 149666,
 'catch': 60574,
 'blocks': 56849,
 'limiting': 105073,
 'simply': 138666,
 'throw': 147411,
 'exceptions': 80337,
 'let': 104091,
 'users': 152828,
 'handle': 91654,
 'them': 147119,
 'new': 114381,
 'idea': 94577,
 'handler': 91762,
 'wrapper': 157672,
 'class': 62620,
 'technical': 145885,
 'information': 96264,
 'value': 153486,
 'operating': 118416,
 'system': 144776,
 'irrelevant': 98343,
 'architecture': 50753,
 'javaessentials': 99482,
 'version': 154098,
 'beta': 55827,
 'subpasses': 143546,
 'msl': 112156,
 'hlsl': 93042,
 'sorry': 140157,
 'issues': 98873,
 'given': 89695,
 'following': 85179,
 'glsl': 89926,
 '450': 27462,
 'core': 66500,
 'extension': 81111,
 'gl_arb_separate_shader_objects': 89717,
 'enable': 78220,
 'layout': 103568,
 'input_attachment_index': 96701,
 'set': 136929,
 'binding': 56310,
 'uniform': 151452,
 'subpassinput': 143547,
 'test': 14

In [81]:
import json
with open('encoder_vocab.json', 'w') as file:
        json.dump(vocab, file, indent=2)

In [73]:
tf.keras.models.save_model(model,"model.keras")
    

In [16]:
model=RandomForestClassifier()

In [19]:
model.fit(X_train,Y_train)


RandomForestClassifier()

In [22]:
import pickle

# save the model to disk
filename = 'rf.pkl'
pickle.dump(model, open(filename, 'wb')) 

In [24]:
pred=model.predict(X_train)

0.7468333333333333

In [85]:
voc=list(LabelEncoder.classes_)

In [86]:
import json
with open('lable_vocab.json', 'w') as file:
        json.dump(voc, file, indent=2)

In [91]:
from flask import Flask, request
import cv2
import numpy as np
import json
import requests
import os
from flask_cors import CORS
import tensorflow
from sklearn.feature_extraction.text import CountVectorizer


In [25]:
app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})
global tokenizer
with open("vocab.json") as f:
    vocab=json.load(f)
tokenizer=CountVectorizer(vocabulary=vocab)
global model
model=tf.keras.models.load_model("model.keras")
global decode_arr
with open("lable_vocab.json") as v:
    vocab==json.load(v)global model
model=tf.keras.models.load_model("model.keras")
global decode_arr
with open("lable_vocab.json") as v:
    vocab==json.load(v)
@app.route("/")
def index():
    return "Hello from Flask!"
def preprocess(desc)
    global tokenizer
    return tokenizer.transform(tokenizer,desc)    
def decode(a)
    global decode_arr
    return decode_arr[a]
@app.route("/api/get_tag", methods=["POST"])
def tagger():
    global model
    data = request.form
    description = data["description"]
    return {"verified": verified}, 200
if __name__ == "__main__":
    app.run(port=6060, debug=True)

SyntaxError: invalid syntax (2973100956.py, line 11)

In [94]:
import flask

[]

In [14]:
with open("vocab.json") as f:
    vocab=json.load(f)

In [168]:
a=CountVectorizer(vocabulary=vocab)

In [ ]:
with open("vocab.json") as f:
    vocab=json.load(f)

In [111]:
with open("lable_vocab.json") as v:
    a=json.load(v)

In [13]:
import json

In [2]:
import tensorflow as tf
model=tf.keras.models.load_model("model.keras")

2023-10-21 04:20:10.458626: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 04:20:10.965943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38402 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [6]:
a=tf.lite.Optimize.OPTIMIZE_FOR_LATENCY

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpemrwcarf/assets


INFO:tensorflow:Assets written to: /tmp/tmpemrwcarf/assets
2023-10-21 04:25:41.773742: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-21 04:25:41.773835: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-21 04:25:41.774476: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpemrwcarf
2023-10-21 04:25:41.776986: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-10-21 04:25:41.777004: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpemrwcarf
2023-10-21 04:25:41.800969: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-10-21 04:25:41.802549: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-10-21 04:25:41.972633: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [169]:
string=["code is crashing and i need help"]

In [180]:
int=a.transform(string)

AttributeError: 'numpy.ndarray' object has no attribute 'transform'

In [189]:
input_details[]

[{'name': 'serving_default_input_9:0',
  'index': 0,
  'shape': array([     1,      1, 166835], dtype=int32),
  'shape_signature': array([    -1,     -1, 166835], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [171]:
model=tf.lite.Interpreter("model.tflite")

In [172]:
model.allocate_tensors()

In [173]:
input_details = model.get_input_details()
output_details = model.get_output_details()
model.set_tensor(0,[description.toarray().astype('float32')])
model.invoke()
output_data = model.get_tensor(output_details[0]['index'])
np.argmax(output_data)

1

In [175]:
input_details[0]['index']

0

In [179]:
a.astype('float32').shape

(1, 166835)

In [182]:
input_details

[{'name': 'serving_default_input_9:0',
  'index': 0,
  'shape': array([     1,      1, 166835], dtype=int32),
  'shape_signature': array([    -1,     -1, 166835], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [188]:
np.array([a]).shape

(1, 1, 166835)